In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install catboost

In [0]:
%load_ext autoreload
%autoreload 2
main_dir = r'/content/drive/My Drive/Kaggle'

data_path = r'/content/drive/My Drive/Kaggle/Data/final_fe'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *

import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Code to make pipeline node

In [0]:
X = pd.read_pickle(f'{data_path}/X.pkl')
test= pd.read_pickle(f'{data_path}/test.pkl')
y= pd.read_pickle(f'{data_path}/y.pkl')

In [0]:
from sklearn.model_selection import GroupKFold

NFOLDS = 6
folds = GroupKFold(n_splits=NFOLDS)


In [0]:
import yaml
categorical_features = yaml.load(open(f'{data_path}/categorical_columns.yaml','r'))
numerical_features = yaml.load(open(f'{data_path}/numerical_columns.yaml','r'))
bad_features = yaml.load(open(f'{data_path}/bad_for_training_features.yaml','r'))

categorical_features = [f for f in categorical_features if f not in bad_features]
all_features =  [f for f in X.columns if f not in bad_features]

In [0]:
from BayDS.lib.training import *

In [0]:
important_features = pd.read_pickle(f'{data_path}/important_features.pkl')

In [0]:
features_to_drop = [f for f in X.columns if f not in important_features.index]

In [0]:
# X.drop(features_to_drop,axis=1,inplace=True)
# test.drop(features_to_drop,axis=1,inplace=True)
all_features =  [f for f in all_features if f in important_features.index]

In [0]:

categorical_features =  [f for f in categorical_features if f in important_features.index]

In [0]:
important_features.columns

Index(['importance'], dtype='object')

In [0]:
# nans_groups = {}
# temp_df = train.isna()
# temp_df2 = test.isna()

# nans_df = pd.concat([temp_df, temp_df2])

# for col in list(nans_df):
#     cur_group = nans_df[col].sum()
#     if cur_group>0:
#         try:
#             nans_groups[cur_group].append(col)
#         except:
#             nans_groups[cur_group]=[col]

# add_category = []
# for col in nans_groups:
#     if len(nans_groups[col])>1:
#         train['nan_group_'+str(col)] = np.where(temp_df[nans_groups[col]].sum(axis=1)>0,1,0).astype(np.int8)
#         test['nan_group_'+str(col)]  = np.where(temp_df2[nans_groups[col]].sum(axis=1)>0,1,0).astype(np.int8)
#         add_category.append('nan_group_'+str(col))

In [0]:
# total_items = len(X)
# keep_cols = ['isFraud','C3_fq_enc']
# for col in numerical_features:
#         cur_dominator = list(X[col].fillna(-999).value_counts())[0]
#         if (cur_dominator/total_items > 0.85) and (col not in keep_cols):
#             cur_dominator = X[col].fillna(-999).value_counts().index[0]
#             print('Column:', col, ' | Dominator:', cur_dominator)
#             X[col] = np.where(X[col].fillna(-999)==cur_dominator,1,0)
#             test[col] = np.where(test[col].fillna(-999)==cur_dominator,1,0)

#             X[col] = X[col].fillna(-999).astype(int)
#             test[col] = test[col].fillna(-999).astype(int)

#             if col not in categorical_features:
#                 categorical_features.append(col)

In [0]:
params = {
#                 'n_estimators':5000,
                'learning_rate': 0.01,
                'eval_metric':'AUC',
                'loss_function':'Logloss',
                'random_seed':42,
                'metric_period':500,
                'od_wait':500,
                'task_type':'GPU',
                'depth': 8,
                #'colsample_bylevel':0.7,
                } 
n_fold = 6
folds = GroupKFold(n_splits=n_fold)
train_options = {
    "model_type":'cat',
    "params": params,
    'early_stopping_rounds': 1000,
    'n_estimators': 15000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
    
}

In [0]:
result_dict_cb_new_id = train_model_classification_vb(X=X, X_test=test, columns=all_features,
                                                   y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             categorial_columns = categorical_features,
                                             plot_feature_importance=True,
                                             verbose=100, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=X['DT_M'])


Fold 1 started at Mon Sep 30 05:13:56 2019
0:	learn: 0.9114607	test: 0.7821377	best: 0.7821377 (0)	total: 571ms	remaining: 2h 22m 44s
500:	learn: 0.9670880	test: 0.8775412	best: 0.8775412 (500)	total: 3m 36s	remaining: 1h 44m 20s
1000:	learn: 0.9735100	test: 0.9055785	best: 0.9055785 (1000)	total: 7m 26s	remaining: 1h 43m 58s


In [0]:
import pickle
pickle.dump(result_dict_cb_new_id,open(f'{data_path}/results_dict_catboost.pkl','wb'))

In [0]:
pd.DataFrame(result_dict_cb_new_id['oof'], columns=['isFraud'], index=X.index).to_csv(f'{data_path}/oof_all_data_best_lgb.csv')

pd.DataFrame(result_dict_cb_new_id['prediction'], columns=['isFraud'], index=test.index).to_csv(f'{data_path}/prediction_all_data_best_lgb.csv')

sample_submission = pd.read_csv(data_path + '/sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_dict_cb_new_id['prediction'], columns=['isFraud'], index=test.index)

sample_submission['isFraud'] = sub1#*0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv(f'{data_path}/catfinal_sub.csv')